In [ ]:
from pathlib import Path
from transition_probability_estimation import *
from Data_synthesize import *
from rw_data_processing import *
import seaborn
import matplotlib.pyplot as plt
import numpy as np
from plot_results import *
%load_ext autoreload
%autoreload 2
plt.style.use("rw_visualization.mplstyle")


In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = seaborn.color_palette()
current_palette


# 1. Course of disease synthesis

In [18]:
# Load firfly parameters
folder = Path('./firefly_result/Firefly_result_pop_size_100_alpha_1_betamin_1_gamma_0.131_max_generations_200')

file_name = 'firefly_best.txt'
with open(folder/file_name, 'r') as f:
    results = [[float(num) for num in line.split(' ')] for line in f]
results = np.array(results)
best_index = np.argmin(results[:, -1])
best_firefly = results[best_index, 1:-1]
input_P = best_firefly

In [19]:
# Load firfly parameters
file_name = 'firefly_best.txt'
with open(folder/file_name, 'r') as f:
    results = [[float(num) for num in line.split(' ')] for line in f]
results = np.array(results)
best_index = np.argmin(results[:, -1])
best_firefly = results[best_index, 1:-1]
input_P = best_firefly

# Load course of disease data
overdispersion_rate = input_P[35]
overdispersion_weight = input_P[36]
latent_period_gamma = {
    'latent_period_shape': input_P[37], 'latent_period_scale': input_P[38]}
infectious_period_gamma = {
    'infectious_period_shape': input_P[39], 'infectious_period_scale': input_P[40]}
incubation_period_gamma = {
    'incubation_period_shape': input_P[41], 'incubation_period_scale': input_P[42]}
symptom_to_isolation_gamma = {
    'symptom_to_confirmed_shape': input_P[43], 'symptom_to_confirmed_scale': input_P[44], 'symptom_to_confirmed_loc': input_P[45]}
asymptomatic_to_recovered_gamma = {
    'asymptomatic_to_recovered_shape': input_P[46], 'asymptomatic_to_recovered_scale': input_P[47], 'asymptomatic_to_recovered_loc': input_P[48]}
symptomatic_to_critically_ill_gamma = {
    'symptomatic_to_critically_ill_shape': input_P[49], 'symptomatic_to_critically_ill_scale': input_P[50], 'symptomatic_to_critically_ill_loc': input_P[51]}
symptomatic_to_recovered_gamma = {
    'symptomatic_to_recovered_shape': input_P[52], 'symptomatic_to_recovered_scale': input_P[53], 'symptomatic_to_recovered_loc': input_P[54]}
critically_ill_to_recovered_gamma = {
    'critically_ill_to_recovered_shape': input_P[55], 'critically_ill_to_recovered_scale': input_P[56], 'critically_ill_to_recovered_loc': input_P[57]}
infection_to_death_gamma = {
    'infection_to_death_shape': input_P[58], 'infection_to_death_scale': input_P[59]}
negative_to_confirmed_gamma = {
    'negative_to_confirmed_shape': input_P[60], 'negative_to_confirmed_scale': input_P[61], 'negative_to_confirmed_loc': input_P[62]}
age_risk_ratios = input_P[63:67]
age_risk_ratios = np.repeat(age_risk_ratios, [20, 20, 20, 41])
natural_immunity_rate = input_P[67]
vaccination_rate = input_P[68]
vaccine_efficacy = input_P[69]
attack_rate = {}
attack_rate['household_attack_rate'] = input_P[70:95]
attack_rate['school_attack_rate'] = input_P[95:120]
attack_rate['workplace_attack_rate'] = input_P[120:145]
attack_rate['health_care_attack_rate'] = input_P[145:170]
attack_rate['municipality_attack_rate'] = input_P[170:195]
infection_to_recovered_transition_p = input_P[195]
symptom_to_recovered_transition_p = input_P[196]
critically_ill_to_recovered_transition_p = input_P[197]
transition_p = [infection_to_recovered_transition_p,
                symptom_to_recovered_transition_p, critically_ill_to_recovered_transition_p]

In [20]:
# Courses of disease synthesize
np.random.seed(0)
simulation_size = 10_000
sim_course_of_disease_matrix = np.empty([0, 5])*np.nan
sim_confirmed_matrix = np.empty([0, 3])*np.nan
sim_course_of_disease_data = \
    Draw_course_of_disease_data(0, latent_period_gamma, infectious_period_gamma, 
                                incubation_period_gamma, symptom_to_isolation_gamma,
                                asymptomatic_to_recovered_gamma, symptomatic_to_critically_ill_gamma, 
                                symptomatic_to_recovered_gamma, critically_ill_to_recovered_gamma, 
                                infection_to_death_gamma, negative_to_confirmed_gamma, 
                                natural_immunity_rate, transition_p)
infectious_periods = []
for i in range(simulation_size):
    monitor_isolation_period, latent_period, incubation_period, infectious_period, date_of_critically_ill, \
        date_of_recovery, date_of_death, date_of_confirmed = sim_course_of_disease_data.draw_course_of_disease()
    sim_course_of_disease_matrix = np.vstack([sim_course_of_disease_matrix, 
                                              np.array([latent_period, incubation_period, \
                                                        date_of_critically_ill, date_of_recovery, 
                                                        date_of_death])])
    sim_confirmed_matrix = np.vstack([sim_confirmed_matrix, np.array([monitor_isolation_period,
                                                                      incubation_period,
                                                                      date_of_confirmed])])
    infectious_periods.append(infectious_period)


In [ ]:
_ = plt.hist(infectious_periods, bins=100)

In [22]:
(sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
 sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
 sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
 sim_critically_ill_to_recovered_each_days, 
 sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_matrix)


# 2. Load Taiwan data

In [23]:
# Load data
taiwan_data_path = Path('./data/structured_course_of_disease_data/figshare_taiwan_covid.xlsx')
Taiwan_data_sheet = pd.read_excel(taiwan_data_path, sheet_name=0)
Taiwan_data_sheet.columns = Taiwan_data_sheet.columns.str.strip().str.lower().str.\
    replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [24]:
# Clean data
data_1_to_579 = clean_taiwan_data(Taiwan_data_sheet, 1, 579)

In [25]:
asymptomatic_to_symptom_days = extract_state_data(
    data_1_to_579, 'earliest_infection_date', 'onset_of_symptom')
asymptomatic_to_recover_days = extract_state_data(data_1_to_579, 'earliest_infection_date', 'recovery',
                                                  'onset_of_symptom')
symptom_to_icu_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'icu')
symptom_to_recover_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'recovery', 'icu')
    
icu_to_recover_days = extract_state_data(
    data_1_to_579, 'icu', 'recovery')
icu_to_dead_days = extract_state_data(
    data_1_to_579, 'icu', 'death_date')

asymptomatic_to_confirmed_days = extract_state_data(
    data_1_to_579, 'earliest_infection_date', 'confirmed_date', 'onset_of_symptom')
asymptomatic_to_confirmed_days = asymptomatic_to_confirmed_days.drop(asymptomatic_to_recover_days.index)
symptomatic_to_confirmed_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'confirmed_date')
symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_icu_days.index)
symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_recover_days.index)

In [26]:
# Transition probability
# Stay in asymptomatic
sim_stay_asymptomatic_p = sim_asymptomatic_each_days[1::]/sim_asymptomatic_each_days[0:-1]

# Asymptomatic to symptomatic
sim_asymptomatic_to_symptomatic_p = sim_asymptomatic_to_symptomatic_each_days / \
    sim_asymptomatic_each_days[0:len(sim_asymptomatic_to_symptomatic_each_days)]

# Asymptomatic to recovered
sim_asymptomatic_to_recovered_p = sim_asymptomatic_to_recovered_each_days / \
    sim_asymptomatic_each_days[0:len(sim_asymptomatic_to_recovered_each_days)]

# Stay in symptomatic
sim_stay_symptomatic_p = sim_symptomatic_each_days[1::]/sim_symptomatic_each_days[0:-1]

# Symptomatic to critically ill
sim_symptomatic_to_critically_ill_p = sim_symptomatic_to_critically_ill_each_days / \
    sim_symptomatic_each_days[0:len(sim_symptomatic_to_critically_ill_each_days)]

# Symptomatic to recovered
sim_symptomatic_to_recovered_p = sim_symptomatic_to_recovered_each_days / \
    sim_symptomatic_each_days[0:len(sim_symptomatic_to_recovered_each_days)]

# Stay in critically ill
sim_stay_critically_ill_p = sim_critically_ill_each_days[1::] / \
    sim_critically_ill_each_days[0:-1]

# Critically ill to recovered
sim_critically_ill_to_recovered_p = sim_critically_ill_to_recovered_each_days / \
    sim_critically_ill_each_days[0:len(sim_critically_ill_to_recovered_each_days)]

# Critically ill to death
sim_critically_ill_to_death_p = sim_critically_ill_to_death_each_days / \
    sim_critically_ill_each_days[0:len(sim_critically_ill_to_death_each_days)]


In [ ]:
plt.plot(sim_critically_ill_to_recovered_p)
plt.xlabel('Day')
plt.ylabel('Probability')
plt.title('Critically ill to recovered')


# 3. Compare synthetic data to Taiwan data.

## 3.1 Asymptomatic to symptomatic

In [ ]:
sim_asymptomatic_to_symptomatic = np.flip(
    np.cumsum(np.flip(sim_asymptomatic_to_symptomatic_each_days)))

_ = state_transition_plot(asymptomatic_to_symptom_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', 
                          save_fig=False)
plt.stairs(sim_asymptomatic_to_symptomatic/sim_asymptomatic_to_symptomatic[0],
           edges=np.arange(0, len(sim_asymptomatic_to_symptomatic)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_symptomatic[0])})')
plt.xlim([-1, 80])
plt.legend(fontsize=18)

## 3.2 Asymptomatic to recovered

In [ ]:
sim_asymptomatic_to_recovered = np.flip(
    np.cumsum(np.flip(sim_asymptomatic_to_recovered_each_days)))

_ = state_transition_plot(asymptomatic_to_recover_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', 
                          save_fig=False)
plt.stairs(sim_asymptomatic_to_recovered/sim_asymptomatic_to_recovered[0],
           edges=np.arange(0, len(sim_asymptomatic_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_recovered[0])})')
plt.xlim([-1, 80])
plt.legend(fontsize=18)

## 3.3 Asymptomatic to confirmed

In [30]:
# sim_confirmed_matrix: [monitor_isolation_period, incubation_period, date_of_critically_ill]
IA_2_C_days = 0
for row in sim_confirmed_matrix:
    if np.isnan(row[1]):
        IA_2_C_days = max(IA_2_C_days, row[0])

sim_asymptomatic_to_confirmed_each_days = np.zeros(int(IA_2_C_days)+1)
for row in sim_confirmed_matrix:
    if np.isnan(row[1]):
        sim_asymptomatic_to_confirmed_each_days[int(row[0])] += 1

In [ ]:
sim_asymptomatic_to_confirmed = np.flip(
    np.cumsum(np.flip(sim_asymptomatic_to_confirmed_each_days)))

_ = state_transition_plot(asymptomatic_to_confirmed_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{C}$', 
                          save_fig=False)
plt.stairs(sim_asymptomatic_to_confirmed/sim_asymptomatic_to_confirmed[0],
           edges=np.arange(0, len(sim_asymptomatic_to_confirmed)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_confirmed[0])})')
plt.xlim([-1, 80])
plt.legend(fontsize=18)

## 3.4 Symptomatic to critically ill

In [32]:
sim_symptomatic_to_critically_ill = np.flip(
    np.cumsum(np.flip(sim_symptomatic_to_critically_ill_each_days)))


In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)


# Symptom to ICU
cumsymptom_to_icu_days = state_transition_plot(
    symptom_to_icu_days, 'Symptomatic', 'critically ill', save_fig=False)
ax.stairs(sim_symptomatic_to_critically_ill/sim_symptomatic_to_critically_ill[0],
           edges=np.arange(0, len(sim_symptomatic_to_critically_ill)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_critically_ill[0])})')
ax.set_xlim([-1, 80])

# ax.set_xlabel('Day')
ax.set_ylabel('Proportion of cases')
ax.legend(fontsize=18)

# fig.savefig('RW2022_state_transition_verification_symptom_to_icu.pdf')

## 3.5 Symptomatic to recovered
  

In [34]:
sim_symptomatic_to_recovered = np.flip(
    np.cumsum(np.flip(sim_symptomatic_to_recovered_each_days)))


In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
# Symptom to recovered
cumsymptom_to_recover_days = state_transition_plot(
    symptom_to_recover_days, 'Symptomatic', 'recovered', save_fig=False)
ax.stairs(sim_symptomatic_to_recovered/sim_symptomatic_to_recovered[0],
           edges=np.arange(0, len(sim_symptomatic_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_recovered[0])})')
ax.set_xlim([-1, 80])

# ax.set_xlabel('Day')
# ax.set_ylabel('Proportion of cases')
ax.legend(fontsize=18)

# fig.savefig('RW2022_state_transition_verification_symptom_to_recover.pdf')

## 3.6 Symptomatic to confirmed

In [36]:
# sim_confirmed_matrix: [monitor_isolation_period, incubation_period, date_of_critically_ill]
IS_2_C_days = 0
for row in sim_confirmed_matrix:
    if ~np.isnan(row[1]):
        IS_2_C_days = max(IS_2_C_days, row[0]-row[1])

sim_symptomatic_to_confirmed_each_days = np.zeros(int(IS_2_C_days)+1)
for row in sim_confirmed_matrix:
    if ~np.isnan(row[1]):
        sim_symptomatic_to_confirmed_each_days[int(row[0]-row[1])] += 1

In [ ]:
sim_symptomatic_to_confirmed = np.flip(
    np.cumsum(np.flip(sim_symptomatic_to_confirmed_each_days)))

_ = state_transition_plot(symptomatic_to_confirmed_days, 
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{C}$', 
                          save_fig=False)
plt.stairs(sim_symptomatic_to_confirmed/sim_symptomatic_to_confirmed[0],
           edges=np.arange(0, len(sim_symptomatic_to_confirmed)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_confirmed[0])})')
plt.xlim([-1, 80])
plt.legend(fontsize=18)

## 3.7 Critically ill to recovered

In [38]:
sim_critically_ill_to_recovered = np.flip(
    np.cumsum(np.flip(sim_critically_ill_to_recovered_each_days)))


In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)

cumicu_to_recover_days =  state_transition_plot(
    icu_to_recover_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{R}$', save_fig=False)
ax.stairs(sim_critically_ill_to_recovered/sim_critically_ill_to_recovered[0],
           edges=np.arange(0, len(sim_critically_ill_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_critically_ill_to_recovered[0])})')
ax.set_xlim([-1, 80])

ax.set_xlabel('Day')
ax.set_ylabel('Proportion of cases')
ax.legend(fontsize=18)

# fig.savefig('RW2022_state_transition_verification_icu_to_recover.pdf')

## 3.8 Crtically ill to death

In [40]:
sim_critically_ill_to_death = np.flip(
    np.cumsum(np.flip(sim_critically_ill_to_death_each_days)))


In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)

cumicu_to_dead_days = state_transition_plot(
    icu_to_dead_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{D}$', save_fig=False)
ax.stairs(sim_critically_ill_to_death/sim_critically_ill_to_death[0],
           edges=np.arange(0, len(sim_critically_ill_to_death)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_critically_ill_to_death[0])})')
ax.set_xlim([-1, 80])

ax.set_xlabel('Day')
# ax.set_ylabel('Proportion of cases')
ax.legend(fontsize=18)

# fig.savefig('RW2022_state_transition_verification_icu_to_dead.pdf')

## 3.9 Combine plots

In [ ]:
fig = plt.figure(figsize=(18, 18)) # Adjust the size as needed
ax1 = fig.add_subplot(331)

# 3.1 Asymptomatic to symptomatic
_ = state_transition_plot(asymptomatic_to_symptom_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', 
                          save_fig=False)
ax1.stairs(sim_asymptomatic_to_symptomatic/sim_asymptomatic_to_symptomatic[0],
           edges=np.arange(0, len(sim_asymptomatic_to_symptomatic)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_symptomatic[0])})')
ax1.set_xlim([-1, 80])
ax1.set_xlabel('')
ax1.set_ylabel('Proportion of cases')
ax1.legend(fontsize=18)


# 3.2 Asymptomatic to recovered
ax2 = fig.add_subplot(332)
_ = state_transition_plot(asymptomatic_to_recover_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{R}$', 
                          save_fig=False)
ax2.stairs(sim_asymptomatic_to_recovered/sim_asymptomatic_to_recovered[0],
           edges=np.arange(0, len(sim_asymptomatic_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_recovered[0])})')
ax2.set_xlim([-1, 80])
ax2.set_xlabel('')
ax2.set_ylabel('')
# ax2.set_ylabel('Proportion of cases')
ax2.legend(fontsize=18)

# 3.3 Asymptomatic to confirmed
ax3 = fig.add_subplot(333)
_ = state_transition_plot(asymptomatic_to_confirmed_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{C}$', 
                          save_fig=False)
ax3.stairs(sim_asymptomatic_to_confirmed/sim_asymptomatic_to_confirmed[0],
           edges=np.arange(0, len(sim_asymptomatic_to_confirmed)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_asymptomatic_to_confirmed[0])})')

ax3.set_xlim([-1, 80])
ax3.set_xlabel('')
ax3.set_ylabel('')
# ax3.set_ylabel('Proportion of cases')
ax3.legend(fontsize=18)

# 3.4 Symptom to ICU
ax4 = fig.add_subplot(334)
_ = state_transition_plot(symptom_to_icu_days, 
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{I}^\mathrm{C}$', 
                          save_fig=False)
ax4.stairs(sim_symptomatic_to_critically_ill/sim_symptomatic_to_critically_ill[0],
           edges=np.arange(0, len(sim_symptomatic_to_critically_ill)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_critically_ill[0])})')
ax4.set_xlim([-1, 80])
ax4.set_xlabel('')
# ax4.set_ylabel('Proportion of cases')
ax4.legend(fontsize=18)

# 3.5 Symptom to recovered
ax5 = fig.add_subplot(335)
_ = state_transition_plot(symptom_to_recover_days, 
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{R}$', 
                          save_fig=False)
ax5.stairs(sim_symptomatic_to_recovered/sim_symptomatic_to_recovered[0],
           edges=np.arange(0, len(sim_symptomatic_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_recovered[0])})')
ax5.set_xlim([-1, 80])
ax5.set_xlabel('')
ax5.set_ylabel('')
# ax5.set_ylabel('Proportion of cases')
ax5.legend(fontsize=18)

# 3.6 Symptom to confirmed
ax6 = fig.add_subplot(336)
_ = state_transition_plot(symptomatic_to_confirmed_days, 
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{C}$', 
                          save_fig=False)
ax6.stairs(sim_symptomatic_to_confirmed/sim_symptomatic_to_confirmed[0],
           edges=np.arange(0, len(sim_symptomatic_to_confirmed)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_symptomatic_to_confirmed[0])})')
ax6.set_xlim([-1, 80])
ax6.set_ylabel('')
# ax6.set_xlabel('Day')
# ax6.set_ylabel('Proportion of cases')
ax6.legend(fontsize=18)

# 3.7 Critical to recovered
ax7 = fig.add_subplot(337)
_ = state_transition_plot(
    icu_to_recover_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{R}$', save_fig=False)
ax7.stairs(sim_critically_ill_to_recovered/sim_critically_ill_to_recovered[0],
           edges=np.arange(0, len(sim_critically_ill_to_recovered)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_critically_ill_to_recovered[0])})')
ax7.set_xlim([-1, 80])
# ax6.set_xlabel('Day')
# ax6.set_ylabel('Proportion of cases')
ax7.legend(fontsize=18)
ax7.legend(fontsize=18)

# 3.8 Critical to death
ax8 = fig.add_subplot(338)
_ = state_transition_plot(
    icu_to_dead_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{D}$', save_fig=False)
ax8.stairs(sim_critically_ill_to_death/sim_critically_ill_to_death[0],
           edges=np.arange(0, len(sim_critically_ill_to_death)+1)-1, linewidth=2, 
           label=f'Simulation ({int(sim_critically_ill_to_death[0])})')
ax8.set_xlim([-1, 80])

ax8.set_xlabel('Day')
ax8.set_ylabel('')
# ax.set_ylabel('Proportion of cases')
ax8.legend(fontsize=18)


plt.tight_layout()
plt.show()
# fig.savefig('RW2024_state_transition_verification_all.pdf')


# 4. Compare synthetic data to Taiwan data with bootstraping

## 4.1 Define the bootstrap function

In [43]:
def sim_course_bootstrap(sim_course_of_disease_matrix, start_state, end_state, bootstrap_num=100):
    matrix = sim_course_of_disease_matrix.copy()

    idx = []
    if start_state == 'asymptomatic':
        if end_state == 'symptomatic':
            for i, row in enumerate(matrix):
                if ~np.isnan(row[1]):
                    idx.append(i)
        elif end_state == 'recovered':
            for i, row in enumerate(matrix):
                if np.isnan(row[1]) and ~np.isnan(row[3]):
                    idx.append(i)
        else:
            print('end state error')
    # elif start_state == 

    matrix = matrix[idx, :]
    idx = np.random.randint(len(matrix), size=bootstrap_num)
    sim_course_of_disease_matrix_sample = matrix[idx, :]

    return(sim_course_of_disease_matrix_sample)
    

In [44]:
def sim_course_bootstrap(sim_course_of_disease_matrix, start_state, end_state, bootstrap_num=100):
    # sim_course_of_disease_matrix -> [latent_period, incubation_period, date_of_critically_ill, date_of_recovery, date_of_death]
    # Filter rows based on start and end states
    if start_state == 'asymptomatic':
        if end_state == 'symptomatic':
            idx = np.where(~np.isnan(sim_course_of_disease_matrix[:, 1]))[0]
        elif end_state == 'recovered':
            idx = np.where(np.isnan(sim_course_of_disease_matrix[:, 1]) & ~np.isnan(sim_course_of_disease_matrix[:, 3]))[0]
        else:
            raise ValueError('Invalid end state')
    elif start_state == 'symptomatic':
        if end_state == 'recovered':
            idx = np.where(~np.isnan(sim_course_of_disease_matrix[:, 1]) & np.isnan(sim_course_of_disease_matrix[:, 2]) & ~np.isnan(sim_course_of_disease_matrix[:, 3]))[0]
        elif end_state == 'icu':
            idx = np.where(~np.isnan(sim_course_of_disease_matrix[:, 1]) & ~np.isnan(sim_course_of_disease_matrix[:, 2]))[0]
        else:
            raise ValueError('Invalid end state')
    elif start_state == 'icu':
        if end_state == 'recovered':
            idx = np.where(~np.isnan(sim_course_of_disease_matrix[:, 2]) & ~np.isnan(sim_course_of_disease_matrix[:, 3]))[0]
        elif end_state == 'dead':
            idx = np.where(~np.isnan(sim_course_of_disease_matrix[:, 2]) & ~np.isnan(sim_course_of_disease_matrix[:, 4]))[0]
        else:
            raise ValueError('Invalid end state')
    else:
        raise ValueError('Invalid start state')

    # Sample from filtered rows
    sample_indices = np.random.choice(idx, size=bootstrap_num, replace=True)
    sim_course_of_disease_matrix_sample = sim_course_of_disease_matrix[sample_indices]

    return sim_course_of_disease_matrix_sample

In [45]:
def sim_course_confirmed_bootstrap(sim_confirmed_matrix: np.array, start_state: str, bootstrap_num=100):
    '''
    Random sampling by bootstrap.

    Input
    -----
    sim_confirmed_matrix -> [monitor_isolation_period, incubation_period, date_of_confirmed]
    start_state: 'asymptomatic' or 'symptomatic'
    bootstrap_num: number of samples

    Output
    ------
    sim_confirmed_matrix_sample: Random sampled matrix.
    '''
    end_state = 'confirmed'
    matrix = sim_confirmed_matrix.copy()

    idx = []
    if start_state == 'asymptomatic':
        for i, row in enumerate(matrix):
            if np.isnan(row[1]):
                idx.append(i)
    elif start_state == 'symptomatic':
        for i, row in enumerate(matrix):
            if ~np.isnan(row[1]):
                idx.append(i)

    matrix = matrix[idx, :]
    idx = np.random.randint(len(matrix), size=bootstrap_num)
    sim_course_of_disease_matrix_sample = matrix[idx, :]

    return(sim_course_of_disease_matrix_sample)

In [ ]:
sim_course_confirmed_bootstrap(sim_confirmed_matrix, 'asymptomatic', 10)

## 4.2 Visualize the result from bootstraping

* Todo: Merge the bootstraping code to the above KM plots.

In [ ]:
_ = state_transition_plot(asymptomatic_to_symptom_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', 
                          save_fig=False)

In [ ]:
# Asympotomatic to symptomatic
MONTE_NUM = 10
monte_asymptomatic_to_symptomatic = []
max_len = 0
for i in range(MONTE_NUM):
    bootstrap_num = len(asymptomatic_to_symptom_days)
    sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'asymptomatic', 'symptomatic', bootstrap_num)
    (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
    sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
    sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
    sim_critically_ill_to_recovered_each_days, 
    sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
    asymptomatic_to_symptom_days = extract_state_data(
        data_1_to_579, 'earliest_infection_date', 'onset_of_symptom')
    sim_stay_asymptomatic_p = sim_asymptomatic_each_days[1::]/sim_asymptomatic_each_days[0:-1]
    sim_asymptomatic_to_symptomatic = np.flip(
        np.cumsum(np.flip(sim_asymptomatic_to_symptomatic_each_days)))
    monte_asymptomatic_to_symptomatic.append(sim_asymptomatic_to_symptomatic)
    max_len = max(max_len, len(sim_asymptomatic_to_symptomatic))

# Convert unequal len array list to matrix
monte_asymptomatic_to_symptomatic_matrix = np.zeros([len(monte_asymptomatic_to_symptomatic), max_len])
for i, array in enumerate(monte_asymptomatic_to_symptomatic):    
    monte_asymptomatic_to_symptomatic_matrix[i, 0:len(array)] = array

In [49]:
norm_monte_asymptomatic_to_symptomatic = monte_asymptomatic_to_symptomatic_matrix/monte_asymptomatic_to_symptomatic_matrix[0, 0]
norm_monte_mean = np.mean(norm_monte_asymptomatic_to_symptomatic, axis = 0)
ci_ub = np.percentile(norm_monte_asymptomatic_to_symptomatic, 97.5, axis=0)
ci_lb = np.percentile(norm_monte_asymptomatic_to_symptomatic, 2.5, axis=0)


In [ ]:
_ = state_transition_plot(asymptomatic_to_symptom_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', save_fig=False)
plt.stairs(norm_monte_mean,
           edges=np.arange(0, len(norm_monte_mean)+1)-1, linewidth=1, 
           label=f'Simulation ({int(monte_asymptomatic_to_symptomatic_matrix[0, 0])})', color='g')
plt.fill_between(np.arange(len(norm_monte_mean)), ci_lb, ci_ub, color='g', alpha=0.2)
plt.legend(fontsize=18)

In [51]:
# Asymptomatic to confirmed
# Asympotomatic to symptomatic
MONTE_NUM = 10
monte_asymptomatic_to_confirmed = []
max_len = 0
for i in range(MONTE_NUM):
    bootstrap_num = len(asymptomatic_to_confirmed_days)
    sim_confirmed_sample = sim_course_confirmed_bootstrap(sim_confirmed_matrix, 
                                                                  'asymptomatic', 
                                                                  bootstrap_num)
    # sim_confirmed_matrix: [monitor_isolation_period, incubation_period, date_of_critically_ill]
    IA_2_C_days = 0
    for row in sim_confirmed_sample:
        if np.isnan(row[1]):
            IA_2_C_days = max(IA_2_C_days, row[0])

    sim_asymptomatic_to_confirmed_each_days = np.zeros(int(IA_2_C_days)+1)
    for row in sim_confirmed_sample:
        if np.isnan(row[1]):
            sim_asymptomatic_to_confirmed_each_days[int(row[0])] += 1

    sim_asymptomatic_to_confirmed = np.flip(
        np.cumsum(np.flip(sim_asymptomatic_to_confirmed_each_days)))
    monte_asymptomatic_to_confirmed.append(sim_asymptomatic_to_confirmed)
    max_len = max(max_len, len(sim_asymptomatic_to_confirmed))

# Convert unequal len array list to matrix
monte_asymptomatic_to_confirmed_matrix = np.zeros([len(monte_asymptomatic_to_confirmed), max_len])
for i, array in enumerate(monte_asymptomatic_to_confirmed):    
    monte_asymptomatic_to_confirmed_matrix[i, 0:len(array)] = array

In [52]:
norm_monte_asymptomatic_to_confirmed = monte_asymptomatic_to_confirmed_matrix/monte_asymptomatic_to_confirmed_matrix[0, 0]
norm_monte_mean = np.mean(norm_monte_asymptomatic_to_confirmed, axis = 0)
ci_ub = np.percentile(norm_monte_asymptomatic_to_confirmed, 97.5, axis=0)
ci_lb = np.percentile(norm_monte_asymptomatic_to_confirmed, 2.5, axis=0)


In [ ]:
_ = state_transition_plot(asymptomatic_to_confirmed_days, 
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{C}$', save_fig=False)
plt.stairs(norm_monte_mean,
           edges=np.arange(0, len(norm_monte_mean)+1)-1, linewidth=1, 
           label=f'Simulation ({int(monte_asymptomatic_to_confirmed_matrix[0, 0])})', color='g')
plt.fill_between(np.arange(len(norm_monte_mean)), ci_lb, ci_ub, color='g', alpha=0.2)
plt.legend(fontsize=18)

## 4.3 Define Monte Carlo simulation

In [54]:
def mc_sim_course(covid_data: pd.DataFrame, sim_course_of_disease_matrix: np.ndarray, sim_confirmed_matrix: np.ndarray, mc_num=1000):
    '''
    Generate a Monte Carlo simulation matries

    Parameters
    ----------
    covid_data: covid19 individual dataset
    mc_num: number of monte carlo simulation

    Returns
    -------
    mc_matrix: Monte Carlo simulation matries
    '''

    # Initialization
    # sim_course_of_disease_matrix: [monitor_isolation_period, incubation_period, date_of_critically_ill].
    # mc_matrix = []
    
    # Asymptomatic to symptomatic
    max_len = 0
    mc_asymptomatic_to_symptomatic = []
    asymptomatic_to_symptom_days = extract_state_data(
        covid_data, 'earliest_infection_date', 'onset_of_symptom')
    for i in range(mc_num):
        bootstrap_num = len(asymptomatic_to_symptom_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'asymptomatic', 'symptomatic', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_asymptomatic_to_symptomatic = np.flip(
            np.cumsum(np.flip(sim_asymptomatic_to_symptomatic_each_days)))
        mc_asymptomatic_to_symptomatic.append(sim_asymptomatic_to_symptomatic)
        max_len = max(max_len, len(sim_asymptomatic_to_symptomatic))
    # Convert unequal len array list to matrix
    mc_asymptomatic_to_symptomatic_matrix = np.zeros([len(mc_asymptomatic_to_symptomatic), max_len])
    for i, array in enumerate(mc_asymptomatic_to_symptomatic):    
        mc_asymptomatic_to_symptomatic_matrix[i, 0:len(array)] = array

    # Asymptomatic to recovered
    max_len = 0
    mc_asymptomatic_to_recovered = []
    asymptomatic_to_recovered_days = extract_state_data(
        covid_data, 'earliest_infection_date', 'recovery', 'onset_of_symptom')
    for i in range(mc_num):
        bootstrap_num = len(asymptomatic_to_recovered_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'asymptomatic', 'recovered', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_asymptomatic_to_recovered = np.flip(
            np.cumsum(np.flip(sim_asymptomatic_to_recovered_each_days)))
        mc_asymptomatic_to_recovered.append(sim_asymptomatic_to_recovered)
        max_len = max(max_len, len(sim_asymptomatic_to_recovered))
    # Convert unequal len array list to matrix
    mc_asymptomatic_to_recovered_matrix = np.zeros([len(mc_asymptomatic_to_recovered), max_len])
    for i, array in enumerate(mc_asymptomatic_to_recovered): 
        mc_asymptomatic_to_recovered_matrix[i, 0:len(array)] = array

    # Asymptomatic to confirmed
    max_len = 0
    mc_asymptomatic_to_confirmed = []
    asymptomatic_to_confirmed_days = extract_state_data(
        covid_data, 'earliest_infection_date', 'confirmed_date', 'onset_of_symptom')
    asymptomatic_to_confirmed_days = asymptomatic_to_confirmed_days.drop(asymptomatic_to_recovered_days.index)
    for i in range(mc_num):
        bootstrap_num = len(asymptomatic_to_confirmed_days)
        sim_confirmed_sample = sim_course_confirmed_bootstrap(sim_confirmed_matrix, 
                                                                  'asymptomatic', 
                                                                  bootstrap_num)
        IA_2_C_days = 0
        for row in sim_confirmed_sample:
            if np.isnan(row[1]):
                IA_2_C_days = max(IA_2_C_days, row[0])

        sim_asymptomatic_to_confirmed_each_days = np.zeros(int(IA_2_C_days)+1)
        for row in sim_confirmed_sample:
            if np.isnan(row[1]):
                sim_asymptomatic_to_confirmed_each_days[int(row[0])] += 1

        sim_asymptomatic_to_confirmed = np.flip(
            np.cumsum(np.flip(sim_asymptomatic_to_confirmed_each_days)))
        mc_asymptomatic_to_confirmed.append(sim_asymptomatic_to_confirmed)
        max_len = max(max_len, len(sim_asymptomatic_to_confirmed))

    # Convert unequal len array list to matrix
    mc_asymptomatic_to_confirmed_matrix = np.zeros([len(mc_asymptomatic_to_confirmed), max_len])
    for i, array in enumerate(mc_asymptomatic_to_confirmed):    
        mc_asymptomatic_to_confirmed_matrix[i, 0:len(array)] = array


    # Symptomatic to critically ill
    max_len = 0
    mc_symptomatic_to_critically_ill = []
    symptom_to_icu_days = extract_state_data(
        covid_data, 'onset_of_symptom', 'icu')
    for i in range(mc_num):
        bootstrap_num = len(symptom_to_icu_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'symptomatic', 'icu', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_symptomatic_to_icu = np.flip(
            np.cumsum(np.flip(sim_symptomatic_to_critically_ill_each_days)))
        mc_symptomatic_to_critically_ill.append(sim_symptomatic_to_icu)
        max_len = max(max_len, len(sim_symptomatic_to_icu))
    # Convert unequal len array list to matrix
    mc_symptomatic_to_critically_ill_matrix = np.zeros([len(mc_symptomatic_to_critically_ill), max_len])
    for i, array in enumerate(mc_symptomatic_to_critically_ill): 
        mc_symptomatic_to_critically_ill_matrix[i, 0:len(array)] = array

    # Symptomatic to recovered
    max_len = 0
    mc_symptomatic_to_recovered = []
    symptom_to_recover_days = extract_state_data(
        covid_data, 'onset_of_symptom', 'recovery', 'icu')
    for i in range(mc_num):
        bootstrap_num = len(symptom_to_recover_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'symptomatic', 'recovered', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_symptomatic_to_recovered = np.flip(
            np.cumsum(np.flip(sim_symptomatic_to_recovered_each_days)))
        mc_symptomatic_to_recovered.append(sim_symptomatic_to_recovered)
        max_len = max(max_len, len(sim_symptomatic_to_recovered))
    # Convert unequal len array list to matrix
    mc_symptomatic_to_recovered_matrix = np.zeros([len(mc_symptomatic_to_recovered), max_len])
    for i, array in enumerate(mc_symptomatic_to_recovered): 
        mc_symptomatic_to_recovered_matrix[i, 0:len(array)] = array

    # Symptomatic to confirmed
    max_len = 0
    mc_symptomatic_to_confirmed = []
    symptomatic_to_confirmed_days = extract_state_data(
        covid_data, 'onset_of_symptom', 'confirmed_date')
    symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_icu_days.index)
    symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_recover_days.index)
    for i in range(mc_num):
        bootstrap_num = len(symptomatic_to_confirmed_days)
        sim_confirmed_sample = sim_course_confirmed_bootstrap(sim_confirmed_matrix, 
                                                                  'symptomatic', 
                                                                  bootstrap_num)
        IS_2_C_days = 0
        for row in sim_confirmed_sample:
            if ~np.isnan(row[1]):
                IS_2_C_days = max(IS_2_C_days, row[0]-row[1])

        sim_symptomatic_to_confirmed_each_days = np.zeros(int(IS_2_C_days)+1)
        for row in sim_confirmed_sample:
            if ~np.isnan(row[1]):
                sim_symptomatic_to_confirmed_each_days[int(row[0]-row[1])] += 1

        sim_symptomatic_to_confirmed = np.flip(
            np.cumsum(np.flip(sim_symptomatic_to_confirmed_each_days)))
        mc_symptomatic_to_confirmed.append(sim_symptomatic_to_confirmed)
        max_len = max(max_len, len(sim_symptomatic_to_confirmed))

    # Convert unequal len array list to matrix
    mc_symptomatic_to_confirmed_matrix = np.zeros([len(mc_symptomatic_to_confirmed), max_len])
    for i, array in enumerate(mc_symptomatic_to_confirmed):    
        mc_symptomatic_to_confirmed_matrix[i, 0:len(array)] = array

    # Critically ill to recovered
    max_len = 0
    mc_critically_ill_to_recovered = []
    critically_ill_to_recovered_days = extract_state_data(
        covid_data, 'icu', 'recovery')
    for i in range(mc_num):
        bootstrap_num = len(critically_ill_to_recovered_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'icu', 'recovered', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_critically_ill_to_recovered = np.flip(
            np.cumsum(np.flip(sim_critically_ill_to_recovered_each_days)))
        mc_critically_ill_to_recovered.append(sim_critically_ill_to_recovered)
        max_len = max(max_len, len(sim_critically_ill_to_recovered))
    
    # Convert unequal len array list to matrix
    mc_critically_ill_to_recovered_matrix = np.zeros([len(mc_critically_ill_to_recovered), max_len])
    for i, array in enumerate(mc_critically_ill_to_recovered): 
        mc_critically_ill_to_recovered_matrix[i, 0:len(array)] = array
        

    # Critically ill to death
    mc_critically_ill_to_death = []
    critically_ill_to_death_days = extract_state_data(
        covid_data, 'icu', 'death_date')
    for i in range(mc_num):
        bootstrap_num = len(critically_ill_to_death_days)
        sim_course_of_disease_sample = sim_course_bootstrap(sim_course_of_disease_matrix, 'icu', 'dead', bootstrap_num)
        (sim_asymptomatic_each_days, sim_symptomatic_each_days, sim_critically_ill_each_days,
        sim_asymptomatic_to_symptomatic_each_days, sim_asymptomatic_to_recovered_each_days,
        sim_symptomatic_to_critically_ill_each_days, sim_symptomatic_to_recovered_each_days,
        sim_critically_ill_to_recovered_each_days, 
        sim_critically_ill_to_death_each_days) = course_data_to_state_data(sim_course_of_disease_sample)
        sim_critically_ill_to_death = np.flip(
            np.cumsum(np.flip(sim_critically_ill_to_death_each_days)))
        mc_critically_ill_to_death.append(sim_critically_ill_to_death)
    
    # Convert unequal len array list to matrix
    mc_critically_ill_to_death_matrix = np.zeros([len(mc_critically_ill_to_death), max_len])
    for i, array in enumerate(mc_critically_ill_to_death): 
        mc_critically_ill_to_death_matrix[i, 0:len(array)] = array


    return(mc_asymptomatic_to_symptomatic_matrix, mc_asymptomatic_to_recovered_matrix,
            mc_asymptomatic_to_confirmed_matrix, mc_symptomatic_to_critically_ill_matrix,
            mc_symptomatic_to_recovered_matrix, mc_symptomatic_to_confirmed_matrix,
            mc_critically_ill_to_recovered_matrix, mc_critically_ill_to_death_matrix)


In [ ]:
mc_asymptomatic_to_symptomatic_matrix, mc_asymptomatic_to_recovered_matrix,\
    mc_asymptomatic_to_confirmed_matrix, mc_symptomatic_to_critically_ill_matrix,\
    mc_symptomatic_to_recovered_matrix, mc_symptomatic_to_confirmed_matrix,\
    mc_critically_ill_to_recovered_matrix, mc_critically_ill_to_death_matrix= \
    mc_sim_course(data_1_to_579, sim_course_of_disease_matrix, sim_confirmed_matrix, mc_num=1000)

In [56]:
def km_plot_compare(mc_matrix: np.array, tw_days_data: pd.DataFrame, strat_state: str, end_state:str, ax):
    norm_mc_matrix = mc_matrix/mc_matrix[0, 0]
    norm_mc_mean = np.mean(norm_mc_matrix, axis = 0)
    ub = np.percentile(norm_mc_matrix, 97.5, axis=0)
    lb = np.percentile(norm_mc_matrix, 2.5, axis=0)
    _ = state_transition_plot(tw_days_data, strat_state, end_state, save_fig=False)
    ax.stairs(norm_mc_mean,
               edges=np.arange(0, len(norm_mc_mean)+1)-1, linewidth=1, 
               label=f'Simulation', color='g')
    ax.fill_between(np.arange(len(norm_mc_mean)), ub, lb, color='g', alpha=0.2)
    ax.legend(fontsize=18)

In [ ]:
fig = plt.figure(figsize=(18, 18)) # Adjust the size as needed
ax1 = fig.add_subplot(331)

# 3.1 Asymptomatic to symptomatic
km_plot_compare(mc_asymptomatic_to_symptomatic_matrix, asymptomatic_to_symptom_days, '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$', ax1)
ax1.set_xlim([-1, 80])
ax1.set_xlabel('')
ax1.set_ylabel('Proportion of cases')
ax1.legend(fontsize=18)

# 3.2 Asymptomatic to recovered
ax2 = fig.add_subplot(332)
km_plot_compare(mc_asymptomatic_to_recovered_matrix, asymptomatic_to_recover_days, '$\mathrm{I}^\mathrm{A}$', '$\mathrm{R}$', ax2)
ax2.set_xlim([-1, 80])
ax2.set_xlabel('')
ax2.set_ylabel('')
# ax2.set_ylabel('Proportion of cases')
ax2.legend(fontsize=18)

# 3.3 Asymptomatic to confirmed
ax3 = fig.add_subplot(333)
km_plot_compare(mc_asymptomatic_to_confirmed_matrix, asymptomatic_to_confirmed_days, '$\mathrm{I}^\mathrm{A}$', '$\mathrm{C}$', ax3)
ax3.set_xlim([-1, 80])
ax3.set_xlabel('')
ax3.set_ylabel('')
# ax3.set_ylabel('Proportion of cases')
ax3.legend(fontsize=18)

# 3.4 Symptom to ICU
ax4 = fig.add_subplot(334)
km_plot_compare(mc_symptomatic_to_critically_ill_matrix, symptom_to_icu_days, '$\mathrm{I}^\mathrm{S}$', '$\mathrm{I}^\mathrm{C}$', ax4)
ax4.set_xlim([-1, 80])
ax4.set_xlabel('')
# ax4.set_ylabel('Proportion of cases')
ax4.legend(fontsize=18)

# 3.5 Symptom to recovered
ax5 = fig.add_subplot(335)
km_plot_compare(mc_symptomatic_to_recovered_matrix, symptom_to_recover_days, '$\mathrm{I}^\mathrm{S}$', '$\mathrm{R}$', ax5)
ax5.set_xlim([-1, 80])
ax5.set_xlabel('')
ax5.set_ylabel('')
# ax5.set_ylabel('Proportion of cases')
ax5.legend(fontsize=18)

# 3.6 Symptom to confirmed
ax6 = fig.add_subplot(336)
km_plot_compare(mc_symptomatic_to_confirmed_matrix, symptomatic_to_confirmed_days, '$\mathrm{I}^\mathrm{S}$', '$\mathrm{C}$', ax6)
ax6.set_xlim([-1, 80])
ax6.set_ylabel('')
# ax6.set_xlabel('Day')
# ax6.set_ylabel('Proportion of cases')
ax6.legend(fontsize=18)

# 3.7 Critical to recovered
ax7 = fig.add_subplot(337)
km_plot_compare(mc_critically_ill_to_recovered_matrix, icu_to_recover_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{R}$', ax7)
ax7.set_xlim([-1, 80])
# ax6.set_xlabel('Day')
# ax6.set_ylabel('Proportion of cases')
ax7.legend(fontsize=18)
ax7.legend(fontsize=18)

# 3.8 Critical to death
ax8 = fig.add_subplot(338)
km_plot_compare(mc_critically_ill_to_death_matrix, icu_to_dead_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{D}$', ax8)
ax8.set_xlim([-1, 80])
ax8.set_xlabel('Day')
ax8.set_ylabel('')
# ax.set_ylabel('Proportion of cases')
ax8.legend(fontsize=18)

# fig.savefig('RW2024_state_transition_verification_all.pdf')

# 5. Plot day since infection vs days to recovery or death

In [ ]:
# x-axis: Day since infection on y-axis: Days to recovery.
recovered_course = sim_course_of_disease_matrix[np.isnan(
    sim_course_of_disease_matrix[:, -1])]
recovered_days = recovered_course[:, 3]
for i in range(int(recovered_days.max()+1)):
    plt.errorbar(i, np.percentile(recovered_days[recovered_days >= i]-i, 50),
                 yerr=np.array([
                     [np.percentile(recovered_days[recovered_days >= i]-i, 50) -
                      np.percentile(recovered_days[recovered_days >= i]-i, 2.5)],
                     [np.percentile(recovered_days[recovered_days >= i]-i, 97.5)-np.percentile(recovered_days[recovered_days >= i]-i, 50)]]),
                 fmt='k.', ecolor='lightgray', elinewidth=2, capsize=10)

plt.xlabel('Day since infection')
plt.ylabel('Days to recovery')


In [ ]:
# x-axis: Day since infection on y-axis: Days to death.
death_course = sim_course_of_disease_matrix[~np.isnan(
    sim_course_of_disease_matrix[:, -1])]
death_days = death_course[:, -1]
for i in range(int(death_days.max()+1)):
    plt.errorbar(i, np.percentile(death_days[death_days >= i]-i, 50),
                 yerr=np.array([
                     [np.percentile(death_days[death_days >= i]-i, 50) -
                      np.percentile(death_days[death_days >= i]-i, 2.5)],
                     [np.percentile(death_days[death_days >= i]-i, 97.5)-np.percentile(death_days[death_days >= i]-i, 50)]]),
                 fmt='k.', ecolor='lightgray', elinewidth=2, capsize=3)

plt.xlabel('Day since infection')
plt.ylabel('Days to death')


In [ ]:
# x-axis: Day since infection on y-axis: probability of death
for i in range(int(max(max(death_days), max(recovered_days)))):
    plt.plot(i, len(death_days[death_days >= i])/(len(
        death_days[death_days >= i]) + len(recovered_days[recovered_days >= i])), 'k.')

plt.xlabel('Day since infection')
plt.ylabel('Proportion of death cases to recovered and death cases')
